In [1]:
import os


In [2]:
os.chdir("/heavy_data/smkim/workspace/#MES/web-crawler")

In [3]:
os.getcwd()

'/heavy_data/smkim/workspace/#MES/web-crawler'

In [4]:
!python kowiki.py

  0% [                                                ]    4792320 / 1008519338^C
Traceback (most recent call last):
  File "kowiki.py", line 58, in <module>
    filename = wget.download("https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-pages-meta-current.xml.bz2", args.output)
  File "/heavy_data/smkim/.virtualenvs/smkim/lib/python3.6/site-packages/wget.py", line 526, in download
    (tmpfile, headers) = ulib.urlretrieve(binurl, tmpfile, callback)
  File "/usr/lib/python3.6/urllib/request.py", line 277, in urlretrieve
    block = fp.read(bs)
  File "/usr/lib/python3.6/http/client.py", line 463, in read
    n = self.readinto(b)
  File "/usr/lib/python3.6/http/client.py", line 507, in readinto
    n = self.fp.readinto(b)
  File "/usr/lib/python3.6/socket.py", line 586, in readinto
    return self._sock.recv_into(b)
  File "/usr/lib/python3.6/ssl.py", line 1012, in recv_into
    return self.read(nbytes, buffer)
  File "/usr/lib/python3.6/ssl.py", line 874, in read
    return self.

In [10]:
os.getcwd()

'/heavy_data/smkim/workspace/#MES/web-crawler'

In [11]:
import csv

In [12]:
csv.field_size_limit(100000000)

100000000

In [13]:
csv.field_size_limit()

100000000

In [14]:
import pandas as pd

in_file = "kowiki/kowiki_20211230.csv"
out_file = "kowiki.txt"
SEPARATOR = u"\u241D"
df = pd.read_csv(in_file, sep=SEPARATOR, engine="python")
with open(out_file, "w") as f:
    for index, row in df.iterrows():
        f.write(row["text"]) # title 과 text를 중복 되므로 text만 저장 함
        f.write("\n\n\n\n") # 구분자

KeyboardInterrupt: 

In [15]:
import sentencepiece as spm

corpus = "kowiki3.txt"
prefix = "kowiki3"
vocab_size = 3000
spm.SentencePieceTrainer.train(
    f"--input={corpus} --model_prefix={prefix} --vocab_size={vocab_size + 7}" + 
    " --model_type=bpe" +
    " --max_sentence_length=999999" + # 문장 최대 길이
    " --pad_id=0 --pad_piece=[PAD]" + # pad (0)
    " --unk_id=1 --unk_piece=[UNK]" + # unknown (1)
    " --bos_id=2 --bos_piece=[BOS]" + # begin of sequence (2)
    " --eos_id=3 --eos_piece=[EOS]" + # end of sequence (3)
    " --user_defined_symbols=[SEP],[CLS],[MASK]") # 사용자 정의 토큰


In [16]:
import sentencepiece as spm

vocab_file = "kowiki3.model"
vocab = spm.SentencePieceProcessor()
vocab.load(vocab_file)

lines = [
  "겨울이 되어서 날씨가 무척 추워요.",
  "이번 성탄절은 화이트 크리스마스가 될까요?",
  "겨울에 감기 조심하시고 행복한 연말 되세요."
]
for line in lines:
  pieces = vocab.encode_as_pieces(line)
  ids = vocab.encode_as_ids(line)
  print(line)
  print(pieces)
  print(ids)
  print()

겨울이 되어서 날씨가 무척 추워요.
['▁', '겨', '울', '이', '▁', '되어', '서', '▁', '날', '씨', '가', '▁무', '척', '▁추', '워', '요', '.']
[351, 739, 656, 352, 351, 105, 368, 351, 690, 815, 362, 115, 968, 172, 743, 511, 356]

이번 성탄절은 화이트 크리스마스가 될까요?
['▁이', '번', '▁성', '탄', '절', '은', '▁화', '이', '트', '▁크', '리', '스', '마', '스', '가', '▁', '될', '까', '요', '?']
[7, 577, 86, 677, 646, 364, 210, 352, 462, 223, 371, 388, 422, 388, 362, 351, 786, 551, 511, 1199]

겨울에 감기 조심하시고 행복한 연말 되세요.
['▁', '겨', '울', '에', '▁감', '기', '▁조', '심', '하', '시', '고', '▁행', '복', '한', '▁연', '말', '▁되', '세', '요', '.']
[351, 739, 656, 355, 286, 369, 53, 575, 358, 374, 366, 313, 600, 365, 71, 578, 169, 446, 511, 356]



In [17]:
""" configuration json을 읽어들이는 class """
class Config(dict): 
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__

    @classmethod
    def load(cls, file):
        with open(file, 'r') as f:
            config = json.loads(f.read())
            return Config(config)
        
config = Config({
    "n_enc_vocab": len(vocab),
    "n_enc_seq": 256,
    "n_seg_type": 2,
    "n_layer": 6,
    "d_hidn": 256,
    "i_pad": 0,
    "d_ff": 1024,
    "n_head": 4,
    "d_head": 64,
    "dropout": 0.1,
    "layer_norm_epsilon": 1e-12
})
print(config)

{'n_enc_vocab': 3007, 'n_enc_seq': 256, 'n_seg_type': 2, 'n_layer': 6, 'd_hidn': 256, 'i_pad': 0, 'd_ff': 1024, 'n_head': 4, 'd_head': 64, 'dropout': 0.1, 'layer_norm_epsilon': 1e-12}


In [18]:
from torch import nn
""" encoder layer """
class EncoderLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.self_attn = MultiHeadAttention(self.config)
        self.layer_norm1 = nn.LayerNorm(self.config.d_hidn, eps=self.config.layer_norm_epsilon)
        self.pos_ffn = PoswiseFeedForwardNet(self.config)
        self.layer_norm2 = nn.LayerNorm(self.config.d_hidn, eps=self.config.layer_norm_epsilon)
    
    def forward(self, inputs, attn_mask):
        # (bs, n_enc_seq, d_hidn), (bs, n_head, n_enc_seq, n_enc_seq)
        att_outputs, attn_prob = self.self_attn(inputs, inputs, inputs, attn_mask)
        att_outputs = self.layer_norm1(inputs + att_outputs)
        # (bs, n_enc_seq, d_hidn)
        ffn_outputs = self.pos_ffn(att_outputs)
        ffn_outputs = self.layer_norm2(ffn_outputs + att_outputs)
        # (bs, n_enc_seq, d_hidn), (bs, n_head, n_enc_seq, n_enc_seq)
        return ffn_outputs, attn_prob
    
""" encoder """
class Encoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.enc_emb = nn.Embedding(self.config.n_enc_vocab, self.config.d_hidn) #Token embeddings
        self.pos_emb = nn.Embedding(self.config.n_enc_seq + 1, self.config.d_hidn) #position Embeddings
        self.seg_emb = nn.Embedding(self.config.n_seg_type, self.config.d_hidn) #segment Embeddings

        self.layers = nn.ModuleList([EncoderLayer(self.config) for _ in range(self.config.n_layer)])
    
    def forward(self, inputs, segments): #Encoder input에 segment정보 추가 
        positions = torch.arange(inputs.size(1), device=inputs.device, dtype=inputs.dtype).expand(inputs.size(0), inputs.size(1)).contiguous() + 1
        pos_mask = inputs.eq(self.config.i_pad)
        positions.masked_fill_(pos_mask, 0)

        # (bs, n_enc_seq, d_hidn)
        outputs = self.enc_emb(inputs) + self.pos_emb(positions)  + self.seg_emb(segments) #token, position, segment 3가지 embedding +

        # (bs, n_enc_seq, n_enc_seq)
        attn_mask = get_attn_pad_mask(inputs, inputs, self.config.i_pad)

        attn_probs = []
        for layer in self.layers:
            # (bs, n_enc_seq, d_hidn), (bs, n_head, n_enc_seq, n_enc_seq)
            outputs, attn_prob = layer(outputs, attn_mask)
            attn_probs.append(attn_prob)
        # (bs, n_enc_seq, d_hidn), [(bs, n_head, n_enc_seq, n_enc_seq)]
        return outputs, attn_probs

In [19]:
""" bert """
class BERT(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.encoder = Encoder(self.config)

        self.linear = nn.Linear(config.d_hidn, config.d_hidn)
        self.activation = torch.tanh
    
    def forward(self, inputs, segments):
        # (bs, n_seq, d_hidn), [(bs, n_head, n_enc_seq, n_enc_seq)]
        outputs, self_attn_probs = self.encoder(inputs, segments) #Transformer Encoder 
        # (bs, d_hidn)
        outputs_cls = outputs[:, 0].contiguous()
        outputs_cls = self.linear(outputs_cls) # outpdits_cls에 Linear 및 tanh 실행
        outputs_cls = self.activation(outputs_cls)
        # (bs, n_enc_seq, n_enc_vocab), (bs, d_hidn), [(bs, n_head, n_enc_seq, n_enc_seq)]
        return outputs, outputs_cls, self_attn_probs
    
    def save(self, epoch, loss, path):  #Pretrained save
        torch.save({
            "epoch": epoch,
            "loss": loss,
            "state_dict": self.state_dict()
        }, path)
    
    def load(self, path):  #Pretrained load
        save = torch.load(path)
        self.load_state_dict(save["state_dict"])
        return save["epoch"], save["loss"]
    
""" BERT pretrain """
class BERTPretrain(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.bert = BERT(self.config)
        
        # classfier - #BERT의 결과를 입력으로 NSP를 예측하기위한 projection_cls를 선언
        self.projection_cls = nn.Linear(self.config.d_hidn, 2, bias=False) 
        
        # lm - #BERT의 결과를 입력으로 MLM을 예측하기위한 projection_lm을 선언
        self.projection_lm = nn.Linear(self.config.d_hidn, self.config.n_enc_vocab, bias=False)
        
        #projection_lm은 Encoder의 Embedding과 weight를 share 
        self.projection_lm.weight = self.bert.encoder.enc_emb.weight
    
    def forward(self, inputs, segments):
        # (bs, n_enc_seq, d_hidn), (bs, d_hidn), [(bs, n_head, n_enc_seq, n_enc_seq)]
        
        # inputs, segments를 입력으로 BERT를 실행
        outputs, outputs_cls, attn_probs = self.bert(inputs, segments)
        
        # (bs, 2) - # outputs_cls를 입력으로 projection_cls를 실행하여 NSP를 예측
        logits_cls = self.projection_cls(outputs_cls)
        
        # (bs, n_enc_seq, n_enc_vocab) - #outputs를 입력으로 projection_lm을 실행하여 MLM을 예측
        logits_lm = self.projection_lm(outputs)
        
        # (bs, n_enc_vocab), (bs, n_enc_seq, n_enc_vocab), [(bs, n_head, n_enc_seq, n_enc_seq)]
        return logits_cls, logits_lm, attn_probs

In [20]:
import random
""" 마스크 생성 함수 """
def create_pretrain_mask(tokens, mask_cnt, vocab_list): #token을 단어별로 index 배열 행태로 저장 (4~ 10)
    cand_idx = []
    for (i, token) in enumerate(tokens):
        if token == "[CLS]" or token == "[SEP]":
            continue
        if 0 < len(cand_idx) and not token.startswith(u"\u2581"):
            cand_idx[-1].append(i)
        else:
            cand_idx.append([i]) # index를 for loop를 돌며 아래내용(4 ~ 9)을 실행 
    random.shuffle(cand_idx) #Random 선택을 위해 단어의 index shuffle

    mask_lms = []
    for index_set in cand_idx: #mask_lms의 개수가 mask_cnt를 넘지 않도록 (15~ 18)
        if len(mask_lms) >= mask_cnt:#mask_cnt는 전체 token개수의 15%에 해당하는 개수
            break
        if len(mask_lms) + len(index_set) > mask_cnt:
            continue
        for index in index_set:
            masked_token = None
            if random.random() < 0.8: # 80% replace with [MASK] #index에 대해 80% 확률로 [MASK]를 취합니다(21~22)
                masked_token = "[MASK]"
            else:
                if random.random() < 0.5: # 10% keep original #index에 대해 10% 확률로 현재 값을 유지(24~25)
                    masked_token = tokens[index]
                else: # 10% random word                # index에 대해 10% 확률로 vocab_list에서 임의의 값을 선택(26~ 27)
                    masked_token = random.choice(vocab_list)
            mask_lms.append({"index": index, "label": tokens[index]}) #mask된 index의 값과 정답 label을 mask_lms에 저장(28)
            tokens[index] = masked_token #token index의 값을 mask (29)
    mask_lms = sorted(mask_lms, key=lambda x: x["index"]) #Random하게 mask된 값을 index순으로 정렬 (30)
    mask_idx = [p["index"] for p in mask_lms]  # 정렬된 값을 이용해 mask_index, mask_label을 생성 (31~32)
    mask_label = [p["label"] for p in mask_lms]

    return tokens, mask_idx, mask_label

In [21]:
""" 쵀대 길이 초과하는 토큰 자르기 """
def trim_tokens(tokens_a, tokens_b, max_seq):
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_seq:
            break

        if len(tokens_a) > len(tokens_b): #token A의 길이가 길 경우 앞에서부터 토큰을 제거(8~9)
            del tokens_a[0]
        else:                             #token B의 길이가 길 경우 앞에서부터 토큰을 제거(10~11)
            tokens_b.pop()

In [22]:
""" doc별 pretrain 데이터 생성 """ , # 단락별 pretrain 데이터 생성 함수 
from random import randrange
def create_pretrain_instances(docs, doc_idx, doc, n_seq, mask_prob, vocab_list):
    # for [CLS], [SEP], [SEP]
    max_seq = n_seq - 3  #tgt_seq는 n_seq에서 3개를 뺀 값
    tgt_seq = max_seq
    
    instances = []
    current_chunk = []
    current_length = 0
    
    for i in range(len(doc)): #단락을 줄 단위로 for loop를 돌며 아래내용(3 ~ 12)을 실행
        
        current_chunk.append(doc[i]) # line  #current_chunk에 line을 추가, current_length에 라인의 token 수 더하기(11,12)
        current_length += len(doc[i])
        if i == len(doc) - 1 or current_length >= tgt_seq: #마지막 줄 이거나 current_length가 tgt_seq를 넘을 경우 학습데이터 만듦
            if 0 < len(current_chunk):
                a_end = 1                                 #current_chunk에서 Random하게 길이를 선택해서 tokens_a를 만듬(15~20)
                if 1 < len(current_chunk):
                    a_end = randrange(1, len(current_chunk))
                tokens_a = []
                for j in range(a_end):
                    
                    tokens_a.extend(current_chunk[j])
                
                tokens_b = []
                if len(current_chunk) == 1 or random.random() < 0.5:  #50%의 확률로 다른 단락에서 tokens_b를 만듦(23~33)
                    is_next = 0                                 #is_next의 값은 False(0)
                    tokens_b_len = tgt_seq - len(tokens_a)
                    random_doc_idx = doc_idx
                    while doc_idx == random_doc_idx:
                       # print("doc_idx: ", doc_idx)
                        #print("random_doc_idx_before: ", random_doc_idx)
                        random_doc_idx = randrange(0, len(docs))
                        #print("random_doc_idx_after: ", random_doc_idx)
                    random_doc = docs[random_doc_idx]

                    random_start = randrange(0, len(random_doc))
                    for j in range(random_start, len(random_doc)):
                        tokens_b.extend(random_doc[j])
                else:                                          #50%의 확률로 current_chunk에서 tokens_a 이후부터 tokens_b를 만듬(34~37)
                    is_next = 1                                #is_next의 값은 True(1)
                    for j in range(a_end, len(current_chunk)):
                        
                        tokens_b.extend(current_chunk[j])

                trim_tokens(tokens_a, tokens_b, max_seq)         #위에서 정의한 trim_tokens 함수를 실행하여 token 크기를 줄임(39)
                assert 0 < len(tokens_a)
                assert 0 < len(tokens_b)

                tokens = ["[CLS]"] + tokens_a + ["[SEP]"] + tokens_b + ["[SEP]"] #‘[CLS]’ + tokens_a + ‘[SEP]’ + tokens_b + ‘[SEP]’ 형태로 데이터를 생성(43)
                segment = [0] * (len(tokens_a) + 2) + [1] * (len(tokens_b) + 1) #segment를 생성 #윗줄에서 '[CLS]’ + tokens_a + ‘[SEP]’는 0, tokens_b + ‘[SEP]’는 1
                
                tokens, mask_idx, mask_label = create_pretrain_mask(tokens, int((len(tokens) - 3) * mask_prob), vocab_list) #정의한 create_pretrain_mask 함수를 실행하여 Mask 
                                                                                                                           #Mask Token 개수는 전체 Token 수에 0.15(15%)를 곱
                instance = {               #위 결과를 가지고 데이터를 생성 (48~ 55)
                    "tokens": tokens,
                    "segment": segment,
                    "is_next": is_next,
                    "mask_idx": mask_idx,
                    "mask_label": mask_label
                }
                instances.append(instance)

            current_chunk = []
            current_length = 0
    return instances

In [23]:
""" pretrain 데이터 생성 """  #pretrain 데이터 생성함수 
import tqdm
from random import randrange
import random
import json
from tqdm import tqdm_notebook
# ex ) for i in tqdm_notebook(range(10000)) :  ;  print(i)

def make_pretrain_data(vocab, in_file, out_file, count, n_seq, mask_prob):
    vocab_list = []
    for id in range(vocab.get_piece_size()):
        if not vocab.is_unknown(id):
            vocab_list.append(vocab.id_to_piece(id))

    line_cnt = 0
    with open(in_file, "r") as in_f:
        for line in in_f:
            line_cnt += 1
    
    docs = []
    with open(in_file, "r") as f:
        doc = []
        with tqdm_notebook(total=line_cnt, desc=f"Loading") as pbar:
            for i, line in enumerate(f):
                line = line.strip()
                if line == "":
                    if 0 < len(doc):
                        docs.append(doc)
                        doc = []
                else:
                    pieces = vocab.encode_as_pieces(line)
                    if 0 < len(pieces):
                        doc.append(pieces)
                pbar.update(1)
        if doc:
            docs.append(doc)

    for index in range(count):
        output = out_file.format(index)
        if os.path.isfile(output): continue

        with open(output, "w") as out_f:
            with tqdm_notebook(total=len(docs), desc=f"Making") as pbar:
                for i, doc in enumerate(docs):
                    instances = create_pretrain_instances(docs, i, doc, n_seq, mask_prob, vocab_list)
                    for instance in instances:
                        out_f.write(json.dumps(instance))
                        out_f.write("\n")
                    pbar.update(1)

In [24]:
""" pretrain 데이터 생성 실행"""

# 말뭉치 개수(count)는 10로 합니다.
# sequence 길이(n_seq)는 256으로 합니다.
# Mask 확률(mask_prob)는 15%로 합니다.

in_file = "kowiki3.txt"
out_file = "kowiki3_bert_{}.json"
count = 10
n_seq = 256
mask_prob = 0.15

make_pretrain_data(vocab, in_file, out_file, count, n_seq, mask_prob)

/heavy_data/smkim/.virtualenvs/smkim/lib/python3.6/site-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [25]:
import torch
class PretrainDataSet(torch.utils.data.Dataset):
    def __init__(self, vocab, infile):
        self.vocab = vocab
        self.labels_cls = []
        self.labels_lm = []
        self.sentences = []
        self.segments = []

        line_cnt = 0
        with open(infile, "r") as f:
            for line in f:
                line_cnt += 1

        with open(infile, "r") as f:
            for i, line in enumerate(tqdm_notebook(f, total=line_cnt, desc=f"Loading {infile}", unit=" lines")):
                instance = json.loads(line)
                self.labels_cls.append(instance["is_next"])
                sentences = [vocab.piece_to_id(p) for p in instance["tokens"]]
                self.sentences.append(sentences)
                self.segments.append(instance["segment"])
                mask_idx = np.array(instance["mask_idx"], dtype=np.int)
                mask_label = np.array([vocab.piece_to_id(p) for p in instance["mask_label"]], dtype=np.int)
                label_lm = np.full(len(sentences), dtype=np.int, fill_value=-1)
                label_lm[mask_idx] = mask_label
                self.labels_lm.append(label_lm)
    
    def __len__(self):
        assert len(self.labels_cls) == len(self.labels_lm)
        assert len(self.labels_cls) == len(self.sentences)
        assert len(self.labels_cls) == len(self.segments)
        return len(self.labels_cls)
    
    def __getitem__(self, item):
        return (torch.tensor(self.labels_cls[item]),
                torch.tensor(self.labels_lm[item]),
                torch.tensor(self.sentences[item]),
                torch.tensor(self.segments[item]))
    
def pretrin_collate_fn(inputs):
    labels_cls, labels_lm, inputs, segments = list(zip(*inputs))

    labels_lm = torch.nn.utils.rnn.pad_sequence(labels_lm, batch_first=True, padding_value=-1)
    inputs = torch.nn.utils.rnn.pad_sequence(inputs, batch_first=True, padding_value=0)
    segments = torch.nn.utils.rnn.pad_sequence(segments, batch_first=True, padding_value=0)

    batch = [
        torch.stack(labels_cls, dim=0),
        labels_lm,
        inputs,
        segments
    ]
    return batch

In [26]:
""" pretrain 데이터 로더 """
import numpy as np
batch_size = 128
dataset = PretrainDataSet(vocab, "kowiki3_bert_1.json")
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=pretrin_collate_fn)


/heavy_data/smkim/.virtualenvs/smkim/lib/python3.6/site-packages/ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()


In [27]:
""" 모델 epoch 학습 """
def train_epoch(config, epoch, model, criterion_lm, criterion_cls, optimizer, train_loader):
    losses = []
    model.train()

    with tqdm_notebook(total=len(train_loader), desc=f"Train({epoch})") as pbar:
        for i, value in enumerate(train_loader):
            labels_cls, labels_lm, inputs, segments = map(lambda v: v.to(config.device), value)

            optimizer.zero_grad()
            outputs = model(inputs, segments)
            logits_cls, logits_lm = outputs[0], outputs[1]

            loss_cls = criterion_cls(logits_cls, labels_cls)
            loss_lm = criterion_lm(logits_lm.view(-1, logits_lm.size(2)), labels_lm.view(-1))
            loss = loss_cls + loss_lm

            loss_val = loss_lm.item()
            losses.append(loss_val)

            loss.backward()
            optimizer.step()

            pbar.update(1)
            pbar.set_postfix_str(f"Loss: {loss_val:.3f} ({np.mean(losses):.3f})")
    return np.mean(losses)

config.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#print(config)

learning_rate = 5e-5
n_epoch = 20

/heavy_data/smkim/.virtualenvs/smkim/lib/python3.6/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [28]:
import torch.nn.functional as F
""" multi head attention """
class MultiHeadAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.W_Q = nn.Linear(self.config.d_hidn, self.config.n_head * self.config.d_head)
        self.W_K = nn.Linear(self.config.d_hidn, self.config.n_head * self.config.d_head)
        self.W_V = nn.Linear(self.config.d_hidn, self.config.n_head * self.config.d_head)
        self.scaled_dot_attn = ScaledDotProductAttention(self.config)
        self.linear = nn.Linear(self.config.n_head * self.config.d_head, self.config.d_hidn)
        self.dropout = nn.Dropout(config.dropout)
    
    def forward(self, Q, K, V, attn_mask):
        batch_size = Q.size(0)
        # (bs, n_head, n_q_seq, d_head)
        q_s = self.W_Q(Q).view(batch_size, -1, self.config.n_head, self.config.d_head).transpose(1,2)
        # (bs, n_head, n_k_seq, d_head)
        k_s = self.W_K(K).view(batch_size, -1, self.config.n_head, self.config.d_head).transpose(1,2)
        # (bs, n_head, n_v_seq, d_head)
        v_s = self.W_V(V).view(batch_size, -1, self.config.n_head, self.config.d_head).transpose(1,2)

        # (bs, n_head, n_q_seq, n_k_seq)
        attn_mask = attn_mask.unsqueeze(1).repeat(1, self.config.n_head, 1, 1)

        # (bs, n_head, n_q_seq, d_head), (bs, n_head, n_q_seq, n_k_seq)
        context, attn_prob = self.scaled_dot_attn(q_s, k_s, v_s, attn_mask)
        # (bs, n_head, n_q_seq, h_head * d_head)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, self.config.n_head * self.config.d_head)
        # (bs, n_head, n_q_seq, e_embd)
        output = self.linear(context)
        output = self.dropout(output)
        # (bs, n_q_seq, d_hidn), (bs, n_head, n_q_seq, n_k_seq)
        return output, attn_prob
    
""" scale dot product attention """
class ScaledDotProductAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.dropout = nn.Dropout(config.dropout)
        self.scale = 1 / (self.config.d_head ** 0.5)
    
    def forward(self, Q, K, V, attn_mask):
        # (bs, n_head, n_q_seq, n_k_seq)
        scores = torch.matmul(Q, K.transpose(-1, -2))
        scores = scores.mul_(self.scale)
        scores.masked_fill_(attn_mask, -1e9)
        # (bs, n_head, n_q_seq, n_k_seq)
        attn_prob = nn.Softmax(dim=-1)(scores)
        attn_prob = self.dropout(attn_prob)
        # (bs, n_head, n_q_seq, d_v)
        context = torch.matmul(attn_prob, V)
        # (bs, n_head, n_q_seq, d_v), (bs, n_head, n_q_seq, n_v_seq)
        return context, attn_prob
    
""" feed forward """
class PoswiseFeedForwardNet(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.conv1 = nn.Conv1d(in_channels=self.config.d_hidn, out_channels=self.config.d_ff, kernel_size=1)
        self.conv2 = nn.Conv1d(in_channels=self.config.d_ff, out_channels=self.config.d_hidn, kernel_size=1)
        self.active = F.gelu
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, inputs):
        # (bs, d_ff, n_seq)
        output = self.conv1(inputs.transpose(1, 2))
        output = self.active(output)
        # (bs, n_seq, d_hidn)
        output = self.conv2(output).transpose(1, 2)
        output = self.dropout(output)
        # (bs, n_seq, d_hidn)
        return output

In [29]:
""" attention pad mask """
def get_attn_pad_mask(seq_q, seq_k, i_pad):
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    pad_attn_mask = seq_k.data.eq(i_pad)
    pad_attn_mask= pad_attn_mask.unsqueeze(1).expand(batch_size, len_q, len_k)
    return pad_attn_mask

In [30]:

model = BERTPretrain(config)

save_pretrain = "save_bert_pretrain.pth"
best_epoch, best_loss = 0, 0
if os.path.isfile(save_pretrain):
    best_epoch, best_loss = model.bert.load(save_pretrain)
    print(f"load pretrain from: {save_pretrain}, epoch={best_epoch}, loss={best_loss}")
    best_epoch += 1

model.to(config.device)

criterion_lm = torch.nn.CrossEntropyLoss(ignore_index=-1, reduction='mean')
criterion_cls = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

losses = []
offset = best_epoch
for step in range(1,n_epoch):
    epoch = step + offset
    if 0 <= step:
        #del train_loader
        dataset = PretrainDataSet(vocab, f"kowiki3_bert_{epoch % count}.json")
        train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=pretrin_collate_fn)
        print(2)
        print('train_loader:',train_loader)
    loss = train_epoch(config, epoch, model, criterion_lm, criterion_cls, optimizer, train_loader)
    losses.append(loss)
    model.bert.save(epoch, loss, save_pretrain)
    
    
# data
data = {
    "loss": losses
}
df = pd.DataFrame(data)
display(df)

# graph
plt.figure(figsize=[8, 4])
plt.plot(losses)
plt.xlabel('Depth')
plt.xlim((0, n_epoch - 1))
plt.ylabel('Position')
plt.show()

load pretrain from: save_bert_pretrain.pth, epoch=19, loss=12.016347971829502


/heavy_data/smkim/.virtualenvs/smkim/lib/python3.6/site-packages/ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()



2
train_loader: <torch.utils.data.dataloader.DataLoader object at 0x7f6e65242198>


/heavy_data/smkim/.virtualenvs/smkim/lib/python3.6/site-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  



2
train_loader: <torch.utils.data.dataloader.DataLoader object at 0x7f6e6469cb70>


...


2
train_loader: <torch.utils.data.dataloader.DataLoader object at 0x7f6e65278588>



2
train_loader: <torch.utils.data.dataloader.DataLoader object at 0x7f6e65237080>



2
train_loader: <torch.utils.data.dataloader.DataLoader object at 0x7f6e652c8cc0>



2
train_loader: <torch.utils.data.dataloader.DataLoader object at 0x7f6e65237a20>


ValueError: num_samples should be a positive integer value, but got num_samples=0